## 8.3 프리트레인 마친 모델로 문장 생성하기

### .1 모델 초기화

In [1]:
!pip install ratsnlp

In [2]:
# 체크포인트 로드
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [3]:
# 토크나이저 로드
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
# 모델 입력 값 만들기
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

### .2 그리디 서치
- max_length는 생성 최대 길이, 하지만 짧더라도 EOS 토큰 등 스페셜 토큰이 나타나면 생성을 중단
- min_length는 생성 최소길이, 이보다 짧은 구간에서 스페셜 토큰이 등장해 생성이 멈추면 해당 토큰이 나올 확률을 0으로 함
- do_sample은 확률이 높은 단어를 다음 단어로 결정

In [5]:
import torch
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



### .3 빔 서치
- 빔 크기만큼의 선택지를 계산 범위에 넣음. 빔: 경우의 수에서 가장 확률이 높은 시퀀스의 개수

In [6]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


### .4 반복되는 표현 줄이기

In [7]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3, # 토큰이 3개 이상 반복될 경우 3번째 토큰 확률 0으로 변경
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


In [9]:
# 리피티션 페널티 방식
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


### .5 탑-k 샘플링
- 모델이 예측한 다음 토큰 확률 분포에서 확률값이 가장 높은 k개 토큰 가운데 하나를 다음 토큰으로 선택

In [11]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요”라며 “가족과 함께 한 여름 밤의 추억을 선사할 수 있는 더위를 이겨낼 수 있는 색다른 휴가를 준비했다”고 말했다.</d> 서울중앙지법 형사합의24부(성창경 부장판사)는 25일 박


### 템퍼러처 스케일링
- t가 0에 가까워질 수록 토큰 분포가 sharp해진다 > 1등 토큰이 뽑힐 확률이 그만큼 높아진다 > do_sample=True이지만 사실상 greedy decoding이 된다
- 1보더 작게 하면 상대적으로 정확한 문장, 1보다 크게하면 다양한 문장

In [13]:
# 템퍼러처 스케일링
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"제가 뭘 잘못했습니까?"
"우리가 잘못한 게 있습니다, 댁."
"내가 잘못했다, 당신."
세미루는 내 대답을 듣고 당황했지만 그 정도 설명을 듣고 나면 정말


### 탑-p 샘플링
- 확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적 확률값이 p 이상인 최소 개수의 집합 가운데 하나를 다음 토큰으로 선택

In [15]:
# 탑-p 샘플링
# top_p가 0으로 갈 수록 그리디 서치랑 비슷
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요 ㅠㅠ ᅲ
우리가 사랑해요 ᄒᄒ
#미소천사 #좋아요 #선팔맞팔 #선팔환영 #맞팔환영 #인친환


### 종합적용
- 매번 같은 문장이 나올 수 있는 빔 서치, 그리티 서치 대신 샘플링 방식 적용
- 생성된 문장이 너무 짧거나(10) 너무 길지 않게 적용(50)
- 탬퍼러처 스케일링으로 확률 값이 높은 토큰이 더 잘 나오도록
- 탑-k 샘플링과 탑-p샘플링을 동시에 적용해 확률값이 낮은 토큰들은 후보 단어에서 제외

In [16]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?'라고 질문하자 "그거 왜 물어봤나요,"하며 되물었다.
김 씨는 이어 이씨의 대답을 듣고 "이 사람 진짜 뭘 알아봐야할지 모르겠다"고 눈시울이 붉어졌다.
이에 김씨는
